# Numpy Solutions
## Exercise 1: Use Numpy to accelerate the prime sieve

Use Numpy to accelerate the prime sieve

Recall Paul's original Python code

In [ ]:
import  math
def sieve_primes(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

Time how long it takes to find all primes below 10000000 and then write and test a numpy version

In [ ]:
python_time = %timeit -o sieve_primes(10000000)

In [ ]:
import numpy as np
def numpy_sieve(n):
    a = np.ones(n)
    a[0] = 0
    a[1] = 0
    for i in range(2,int(math.sqrt(n))):
         a[i*i::i] = 0
    return np.flatnonzero(a)

In [ ]:
numpy_sieve(100)

In [ ]:
numpy_time = %timeit -o numpy_sieve(10000000)

In [ ]:
print('Numpy is {0:.1f} times faster than pure python'.format(python_time.best/numpy_time.best))

## Exercise 2: Use Numpy to accelerate option pricing

First of all we time the original code. 

In [ ]:
import math # This is the standard Pyton math module. Not the numpy one
import random # This is the standard Python random module.  Not the numpy one

def Asian(so,k,r,v,t,m,n):
    """
    I have not identified what the arguments mean since the original MATLAB code didn't either. 
    This doc-string will be updated once I learn what they are!
    """
    dt = t/m
    AsianPayoffSum = 0
    for i in range(1,n+1):
        s = so
        stSum = so
        at = so
        for j in range(1,m+1):
            st = s * math.exp(((r-v**2/2)*dt) + (v*random.normalvariate(0,1)*math.sqrt(dt)))
            stSum = stSum + st 
            at = stSum/(j+1)
            s = st
        AsianPayoff = max(at-k,0);
        AsianPayoffSum = AsianPayoffSum + AsianPayoff;
    AsianCall = math.exp(-r*t)*(AsianPayoffSum/n)
    return(AsianCall)

In [ ]:
%time Asian(100,90,0.15,0.45,1,100,200000)

Now we rewrite this function using Numpy and discover the speed-up you can reasonably expect.

In [ ]:
#Solution 1 - Removing the inner loop using vectors
import numpy as np

def Asian_numpy(so,k,r,v,t,m,n):
    dt = t/m
    AsianPayoffSum = 0
    for i in range(n):
        st = np.cumprod(np.hstack((so,np.exp(((r-v**2/2)*dt) + (v*np.random.normal(0,1,m)*np.sqrt(dt))))))
        at = np.mean(st)
        AsianPayoff = max(at-k,0);
        AsianPayoffSum = AsianPayoffSum + AsianPayoff;
    AsianCall = np.exp(-r*t)*(AsianPayoffSum/n)
    return(AsianCall)

In [ ]:
#Time it 
%time Asian(100,90,0.15,0.45,1,100,200000)

In [ ]:
# Solution 2 - remove both loops by using matrices
import numpy as np
from numpy import sqrt,exp
from numpy.random import normal

def Asian_numpy2(so,k,r,v,t,m,n):
    dt = t/m
    AsianPayoffSum = 0
    sqrt_dt = sqrt(dt)
    st = np.cumprod( np.hstack((so*np.ones((n,1)),exp( ((r-v**2/2)*dt) + (v*normal(0,1,(n,m)) * sqrt_dt)) )) ,axis=1)
    at = np.mean(st,axis=1)
    AsianPayoff = np.maximum(at-k,0)
    AsianPayoffMean = AsianPayoff.mean()
    AsianCall = np.exp(-r*t)*AsianPayoffMean
    return(AsianCall)

In [ ]:
#See if the result looks about right
Asian_numpy2(100,90,0.15,0.45,1,100,200000)

In [ ]:
#Time it
%time Asian_numpy2(100,90,0.15,0.45,1,100,200000)

If we wanted we could combine this with distributed computing to perform many millions of monte carlo simulations in parallel.

How might you go about doing that?

**Reference**

The original MATLAB code came from here https://sheir.org/mf/asian-option-pricing-using-monte-carlo-simulation-method-in-matlab/

# NumPy tutorial solutions

### Construct the following 7 x 7 matrix using Numpy.

$$
\begin{array}{rrrrrrr}
  0 & 14 & 28 & 42 & 56 & 70 & 84 \\
  2 & 16 & 30 & 44 & 58 & 72 & 86 \\
  4 & 18 & 32 & 46 & 60 & 74 & 88 \\
  6 & 20 & 34 & 48 & 62 & 76 & 90 \\
  8 & 22 & 36 & 50 & 64 & 78 & 92 \\
  10 & 24 & 38 & 52 & 66 & 80 & 94 \\
  12 & 26 & 40 & 54 & 68 & 82 & 96 \\
  \end{array}
$$

In [ ]:
# One solution
a = 2*np.arange(49)
a.shape=(7,7)
a = a.T
a

In [ ]:
# In one line
a = 2*np.arange(49).reshape((7,7)).T
a

In [ ]:
# Another way 
a = 2*np.arange(49).reshape((7,7),order = 'F')
a

### Create a 3x6 array of booleans containing all false values.

In [ ]:
np.full((3,6),False,dtype=np.bool)

Consider the array 

`a = np.arange(20)`

replace every element that's a multiple of 3 with 0

In [ ]:
a = np.arange(20)
a[a % 3==0] = 0
a

Construct the following matrix

$$
\begin{array}{rrrr}
  2 & 0 & 0 & 0 \\
  0 & 2 & 0 & 0  \\
  0 & 0 & 2 & 0  \\
  0 & 0 & 0 & 2  \\
  \end{array}
$$

In [ ]:
np.diag(np.full(4,2))

Construct the following matrix

$$
\begin{array}{rrrr}
  0 & 1 & 0 & 0 \\
  4 & 0 & 1 & 0  \\
  0 & 4 & 0 & 1  \\
  0 & 0 & 4 & 0  \\
  \end{array}
$$

In [ ]:
a = np.diag(np.full(3,1),1)
np.fill_diagonal(a[1:],4)
a